In [1]:
# Import the necessary libraries
%pip install autogluon
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import pickle

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score as acs_score

from src.data.data_fetcher import get_raw_data
from src.features.preprocess_data import get_preprocessed_test_data, fetch_preprocessed_data
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

  Using cached autogluon-0.8.2-py3-none-any.whl (9.7 kB)
  Using cached autogluon.features-0.8.2-py3-none-any.whl (62 kB)
  Using cached autogluon.core-0.8.2-py3-none-any.whl (224 kB)
  Using cached autogluon.timeseries-0.8.2-py3-none-any.whl (116 kB)
  Using cached autogluon.tabular-0.8.2-py3-none-any.whl (285 kB)
  Using cached autogluon.multimodal-0.8.2-py3-none-any.whl (372 kB)
  Using cached autogluon.common-0.8.2-py3-none-any.whl (61 kB)
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached ray-2.3.1-cp310-cp310-win_amd64.whl (21.7 MB)
  Using cached grpcio-1.50.0-cp310-cp310-win_amd64.whl (3.6 MB)
  Using cached pydantic-1.10.13-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached torch-1.13.1-cp310-cp310-win_amd64.whl (162.6 MB)
  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)
  Using cached tensorboard-2.15.1-py3-none-any.whl (5.5 MB)
  Using cached pytorch_lightning-1.9.5-py3-none-any.whl (8

  ERROR: Command errored out with exit status 1:
   command: 'c:\Users\gunna\Documents\Maskinlæring\Prosjekt\power-predictor\venv\Scripts\python.exe' 'c:\Users\gunna\Documents\Maskinlæring\Prosjekt\power-predictor\venv\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' get_requires_for_build_wheel 'C:\Users\gunna\AppData\Local\Temp\tmpr503i8j2'
       cwd: C:\Users\gunna\AppData\Local\Temp\pip-install-fizl45m3\gpustat_5e0839ba47544e939f668b3a026cb986
  Complete output (29 lines):
  Error in sitecustomize; set PYTHONVERBOSE for traceback:
  SyntaxError: (unicode error) 'utf-8' codec can't decode byte 0xe6 in position 0: unexpected end of data (sitecustomize.py, line 7)
  Traceback (most recent call last):
    File "setup.py", line 18, in read_version
      import setuptools_scm
  ModuleNotFoundError: No module named 'setuptools_scm'
  
  During handling of the above exception, another exception occurred:
  
  Traceback (most recent call last):
    File "c:\Users\gunna\Docum

In [2]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
x_test_whole = get_preprocessed_test_data()

x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined, X_train_est_combined, X_val_est_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined, y_train_est_combined, y_val_est_combined])
x_whole.reset_index(drop=True, inplace=True)
y_whole.reset_index(drop=True, inplace=True)


x_whole_obs = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole_obs = pd.concat([y_train_obs_combined, y_val_obs_combined])

x_whole_est = pd.concat([X_train_est_combined, X_val_est_combined])
y_whole_est = pd.concat([y_train_est_combined, y_val_est_combined])

x_whole_est.head(10)

After temporal alignment
X_test_estimated_a.shape = (720, 47), X_test_estimated_b.shape = (720, 47), X_test_estimated_c.shape = (720, 47)
X_test_estimated_a_processed.shape = (720, 61), X_test_estimated_b_processed.shape = (720, 61), X_test_estimated_c_processed.shape = (720, 61)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location_a,location_b,location_c,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,residual_radiation,cloud_ratio,diffuse_cloud_conditional_interaction,cloud_cover_over_30%,sun_addition,direct_rad_cloud_interaction,modified_solar_elevation_squared,is_freezing,is_snow,is_rain
30068,6.900,1.26900,1.654025e+03,0.000000,2871.250000,0.0,278.450012,0.000000,3816.800049,0.000000,0.000000,27.200001,0.000,0.000,0.000,0.00,0.000,0.00,1.00,1020.875000,0.0,0.0,1008.049988,1014.250000,0.000000,0.0,82.650002,1020.450012,0.0,0.0,0.0,239.816254,0.000,280.524994,35.200001,39068.875000,1.100,0.550,0.950,0.0,1,0,0,-0.737943,0.674863,-0.707107,-0.707107,0.000000,0.000000,0.000000,0,0.000000,0.772727,0.0,0,0.000000,0.000000,0.000000,0,0,0
32683,3.775,1.27950,7.385284e+05,163.850006,681.250000,0.0,269.825012,83.225006,318197.500000,42.950001,274824.093750,79.074997,0.000,0.000,0.000,0.00,0.000,1.00,0.00,1014.349976,0.0,0.0,1001.049988,1007.325012,0.000000,0.0,65.500000,1013.599976,0.0,0.0,0.0,224.040497,0.000,273.875000,79.074997,28104.949219,8.000,6.000,-5.300,0.0,1,0,0,0.867099,0.498137,-0.500000,-0.866025,3574.514160,0.213666,0.372124,0,37.675003,1.000000,0.0,1,126.175003,898.728882,0.462240,0,0,0
31298,2.500,1.30150,0.000000e+00,0.000000,1939.224976,0.0,264.625000,0.000000,0.000000,0.000000,0.000000,100.000000,0.150,0.150,0.150,0.15,0.150,0.00,1.00,1010.549988,0.0,0.0,997.349976,1003.650024,0.000000,0.0,59.200001,1010.049988,0.0,0.0,0.3,320.041260,0.100,269.849976,100.000000,11110.724609,5.600,-5.575,0.600,0.0,1,0,0,0.051584,0.998669,-0.707107,0.707107,0.000000,0.000000,0.000000,0,0.000000,1.000000,0.0,1,0.000000,0.000000,0.000000,1,1,0
96801,4.250,1.26900,0.000000e+00,0.000000,389.500000,0.0,271.549988,0.000000,0.000000,0.000000,0.000000,74.199997,0.000,0.000,0.000,0.00,0.000,0.00,1.00,1013.599976,0.0,0.0,998.099976,1004.275024,0.000000,0.0,62.700001,1010.474976,0.0,0.0,0.0,264.199005,0.000,277.500000,87.925003,57794.398438,4.900,-0.975,4.800,0.0,0,0,1,-0.597829,0.801624,-0.965926,-0.258819,0.000000,0.000000,0.000000,0,0.000000,0.843901,0.0,1,0.000000,0.000000,0.000000,0,0,0
65078,4.575,1.28800,0.000000e+00,0.000000,9417.000000,0.0,272.424988,0.000000,0.000000,0.000000,0.000000,47.924999,0.000,0.000,0.000,0.00,0.000,0.00,1.00,1020.700012,0.0,0.0,1007.400024,1013.700012,0.000000,0.0,66.500000,1020.000000,0.0,0.0,0.0,66.585251,0.000,276.799988,95.850006,51880.398438,3.000,-0.100,2.975,0.0,0,1,0,0.608121,0.793844,0.707107,0.707107,0.000000,0.000000,0.000000,0,0.000000,0.500000,0.0,1,0.000000,0.000000,0.000000,0,0,0
65140,3.350,1.27825,0.000000e+00,0.000000,1488.900024,0.0,268.325012,0.000000,0.000000,0.000000,0.000000,100.000000,6.575,1.725,9.075,4.45,4.875,0.00,1.00,998.924988,0.1,3.0,985.650024,991.849976,0.000000,0.0,76.000000,998.150024,0.0,0.0,1.3,261.451508,0.075,271.799988,100.000000,2295.149902,1.925,-1.500,1.150,0.0,0,1,0,0.635068,0.772456,-0.965926,-0.258819,0.000000,0.000000,0.000000,0,0.000000,1.000000,0.0,1,0.000000,0.000000,0.000000,1,1,1
97694,4.625,1.27125,0.000000e+00,0.000000,4061.900146,-1.0,272.475006,0.000000,0.000000,0.000000,0.000000,61.525002,0.000,0.000,0.000,0.00,0.000,0.00,1.00,1002.299988,0.0,0.0,986.950012,993.099976,27.575001,0.0,83.199997,

In [3]:
x_whole["pv_measurement"] = y_whole
df_shuffled = x_whole.sample(frac=1, random_state=42).reset_index(drop=True)
x_whole_a = df_shuffled[df_shuffled['location_a'] == 1]
x_whole_b = df_shuffled[df_shuffled['location_b'] == 1]
x_whole_c = df_shuffled[df_shuffled['location_c'] == 1]

y_whole_a = x_whole_a["pv_measurement"]
x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

y_whole_b = x_whole_b["pv_measurement"]
x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

y_whole_c = x_whole_c["pv_measurement"]
x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)
cat_features = ["time_since_prediction"]

In [4]:
from autogluon.tabular import TabularPredictor

# Define the hyperparameters for CatBoost
catboost_params = {
    'max_depth': 9,
    'cat_features': ['time_since_prediction'],
    'loss_function': 'MAE',
    'verbose': 100
}

# Prepare the hyperparameters for AutoGluon
hyperparameters = {
    'CAT': catboost_params,
    'XGB': {},  # Default parameters for XGBoost
    'GBM': {},  # Default parameters for LightGBM
}

# Initialize the TabularPredictor
best_model_a = TabularPredictor(label='pv_measurement', eval_metric='mean_absolute_error').fit(
    train_data=x_whole_a,
    presets='best_quality',
    hyperparameters=hyperparameters
)


No path specified. Models will be saved in: "AutogluonModels\ag-20231111_154954\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231111_154954\"
AutoGluon Version:  0.8.2
Python Version:     3.10.0
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   11.62 GB / 1022.87 GB (1.1%)
Train Data Rows:    34046
Train Data Columns: 58
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 631.28918, 1166.38771)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass',

In [5]:
from autogluon.tabular import TabularPredictor

# Define the hyperparameters for CatBoost
catboost_params = {
    'max_depth': 9,
    'cat_features': ['time_since_prediction'],
    'loss_function': 'MAE',
    'verbose': 100
}

# Prepare the hyperparameters for AutoGluon
hyperparameters = {
    'CAT': catboost_params,
    'XGB': {},  # Default parameters for XGBoost
    'GBM': {},  # Default parameters for LightGBM
}

# Initialize the TabularPredictor
best_model_b = TabularPredictor(label='pv_measurement', eval_metric='mean_absolute_error').fit(
    train_data=x_whole_b,
    presets='best_quality',
    hyperparameters=hyperparameters,
    time_limit=3600
)


No path specified. Models will be saved in: "AutogluonModels\ag-20231111_155720\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels\ag-20231111_155720\"
AutoGluon Version:  0.8.2
Python Version:     3.10.0
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   11.06 GB / 1022.87 GB (1.1%)
Train Data Rows:    28688
Train Data Columns: 58
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1152.3, 0.0, 96.68964, 204.49334)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binar

KeyboardInterrupt: 

In [ ]:
from autogluon.tabular import TabularPredictor

# Define the hyperparameters for CatBoost
catboost_params = {
    'max_depth': 9,
    'cat_features': ['time_since_prediction'],
    'loss_function': 'MAE',
    'verbose': 100
}

# Prepare the hyperparameters for AutoGluon
hyperparameters = {
    'CAT': catboost_params,
    'XGB': {},  # Default parameters for XGBoost
    'GBM': {},  # Default parameters for LightGBM
}

# Initialize the TabularPredictor
best_model_c = TabularPredictor(label='pv_measurement', eval_metric='mean_absolute_error').fit(
    train_data=x_whole_c,
    presets='best_quality',
    hyperparameters=hyperparameters,
    time_limit=3600
)


In [ ]:
x_whole_a = x_test_whole[x_test_whole['location_a'] == 1]
x_whole_b = x_test_whole[x_test_whole['location_b'] == 1]
x_whole_c = x_test_whole[x_test_whole['location_c'] == 1]

x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)

In [ ]:
y_pred_a = best_model_a.predict(x_whole_a)
y_pred_b = best_model_b.predict(x_whole_b)
y_pred_c = best_model_c.predict(x_whole_c)
y_pred = pd.concat([pd.Series(y_pred_a), pd.Series(y_pred_b), pd.Series(y_pred_c)])
y_pred = y_pred.reset_index(drop=True)
# Save the model
from src.models.saving import save_predictions
from src.features.postprocess_data import postprocess_data

y_pred = postprocess_data(x_test_whole, pd.DataFrame(y_pred))
save_predictions(y_pred, 'autogluon gunnar')